In [17]:
import pandas as pd
import numpy as np
import plotly.express as px

In [66]:
comment_df = pd.read_excel("..\data\cleaned_swed_lemma.xlsx")
comment_df.drop(index = [9, 1555, 2474, 2483, 4346, 7212], inplace = True)

In [67]:
comment_df.drop(index = comment_df[comment_df.author.str.contains("bot", case = False)].index, inplace = True)
comment_df.text = comment_df.text.map(lambda line: " ".join( [word for word in line.split(" ") if (len(word) > 3 or word in ["mzp","dk"])] ))

# Study comment length distribution

In [68]:
lens = comment_df.text.map(lambda line: len(line.split(" ")))
comment_df["lens"] = lens

In [69]:
px.line(y = np.linspace(0,1,len(lens)), x = sorted(lens))

In [70]:
comment_df.drop(columns = comment_df.columns[[0,1]], inplace=True)
comment_df.drop(index = comment_df[comment_df.author.str.contains("bot", case = False)].index, inplace = True)
comment_df.drop(index = comment_df[comment_df.text.str.contains("spam", case = False)].index, inplace = True)
comment_df

,comment_id,text,author,upvotes,timestamp,post_id,lens
0,hgikv58,újranezem Edit elmond vélemény,MarkMew,3,1634150485,q7hft0,4
1,hgilh98,több hallgat dobrev klárából érez iqval rendel...,just_szabi,3,1634150728,q7hft0,8
2,hgiledw,tetszik mzp korrupcióellenes javaslat válaszol...,Buntisteve,2,1634150698,q7hft0,27
3,hgilif7,mzp valós dolog állít kimért óvatos költekezik...,Atekboss,2,1634150741,q7hft0,23
4,hgik5u8,feszült vita hajtépős ideges dobrev lenézően p...,ChronosHD,3,1634150198,q7hft0,16
...,...,...,...,...,...,...,...
8570,hb5zin0,sajtó jelenik híre sosem állított megüt bevert...,hadhazyakos,89,1630486947,pfpjma,28
8571,hb66gf4,protip life bírósági ítéleteket maga egész ért...,orbanygyiktor,7,1630492763,pfpjma,17
8572,hb5zm9h,hangfelvételen dulakodás hamisítvány szegény S...,BitPork,10,1630487037,pfpjma,7
8573,hb643ll,schiffer kavar szar rendes „látlelet garázdasá...,dickblaha,13,1630490921,pfpjma,41


In [71]:
def writeGroupToExcel(min_len, max_len):
    df = comment_df[ comment_df["lens"].between(min_len,max_len) ]
    df.to_csv(f"..\data\group_{min_len}_{max_len}.csv")

writeGroupToExcel(7,10)
writeGroupToExcel(10,20)
writeGroupToExcel(20, 40)
writeGroupToExcel(40, 60)
writeGroupToExcel(60, 100)
writeGroupToExcel(40, 100)
writeGroupToExcel(15, 40)


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

def nlp_summary(min_len, max_len):
    """Returns the number of docs and the volume of the vocabulary"""

    df = comment_df[ comment_df["lens"].between(min_len,max_len-1) ]
    tf_idf_sk = TfidfVectorizer(norm="l1")
    X = tf_idf_sk.fit_transform(df.text)
    return X.shape, df.lens.sum()


nlp_summary(40,100)

((629, 14331), 34600)

In [51]:
comment_df

,comment_id,text,author,upvotes,timestamp,post_id,lens
0,hgikv58,látt újranez elmondt vélemény,MarkMew,3,1634150485,q7hft0,4
1,hgilh98,több hallgat dobrev klár rendelkez szavazn,just_szabi,3,1634150728,q7hft0,6
2,hgiledw,tetszet mzp korrupcióellenes javaslat válaszol...,Buntisteve,2,1634150698,q7hft0,25
3,hgilif7,mzp valós dolg állí kimért óvatos költekez dob...,Atekboss,2,1634150741,q7hft0,22
4,hgik5u8,feszül hajtépős ideges dobrev lenéző péterezet...,ChronosHD,3,1634150198,q7hft0,15
...,...,...,...,...,...,...,...
8590,hb5zin0,sajtó jelent állított megütöt bevert radiátor ...,hadhazyakos,89,1630486947,pfpjma,25
8591,hb66gf4,protip bíróság ítélet egész értelmezn kiragado...,orbanygyiktor,7,1630492763,pfpjma,15
8592,hb5zm9h,hangfelvétel dulakodás hamisítvány szegény szt...,BitPork,10,1630487037,pfpjma,7
8593,hb643ll,schiffer kavarja szar rendes „látlelet” garázd...,dickblaha,13,1630490921,pfpjma,41
